## Analysis of Technical Indicators
> By Srishti Singh
*   Combination of all technical indicators

In [ ]:
# importing the libraries
import pandas as pd
import numpy as np
import yfinance as yfin
from pandas_datareader import data as pdr
import datetime
yfin.pdr_override()
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
# setting the duration of the data to 1 year
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(365)

In [ ]:
def get_data(ticker):
  asset = pdr.get_data_yahoo(ticker + '.NS', start = start_date, end = end_date) #.NS is used for stocks on the NSE
  asset.reset_index(inplace = True)
  return asset

In [ ]:
spicejet = get_data('SPICEJET')
spicejet

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-11-11,39.500000,39.650002,38.849998,39.049999,39.049999,1052965
1,2022-11-14,39.000000,39.200001,38.599998,38.900002,38.900002,1442406
2,2022-11-15,38.000000,39.200001,37.250000,38.700001,38.700001,1857880
3,2022-11-16,38.650002,40.349998,38.299999,39.349998,39.349998,2588773
4,2022-11-17,39.549999,39.549999,38.549999,38.700001,38.700001,867328
...,...,...,...,...,...,...,...
110,2023-04-24,32.450001,32.450001,31.900000,32.000000,32.000000,575827
111,2023-04-25,32.049999,32.299999,26.250000,30.049999,30.049999,2073222
112,2023-04-26,30.700001,32.900002,30.450001,31.200001,31.200001,9813328
113,2023-04-27,31.299999,31.750000,30.900000,31.100000,31.100000,1661410


In [ ]:
# Functions for all the technical indicators from scratch
'''
Moving Average Convergence Divergence (MACD)
Source: https://groww.in/p/what-is-macd
Inputs: asset -> The dataframe of the stock
        long_period -> The long period EWMA (default = 26)
        short_period -> The short period EWMA (default = 12)
        signal -> The period EWMA of the macd line (default = 9)
'''
def macd(asset, period_long = 26, period_short = 12, signal = 9):
  asset['MACDval'] = asset['Close'].ewm(span = period_short).mean() - asset['Close'].ewm(span = period_long).mean()
  asset['MACDsignal'] = asset['MACDval'].ewm(span = signal).mean()
  asset['MACDh'] = asset['MACDval'] - asset['MACDsignal']
  return asset

'''
On-Balance Volume (OBV)
Source: https://www.tradingview.com/support/solutions/43000502593-on-balance-volume-obv/
Inputs: asset -> The dataframe of the stock
'''
def obv(asset):
  obv = [0]
  for i in range(1, len(asset.Close)):
    if asset.Close[i] > asset.Close[i - 1]:
      obv.append(obv[-1] + asset.Volume[i])
    elif asset.Close[i] < asset.Close[i - 1]:
      obv.append(obv[-1] - asset.Volume[i])
    else:
      obv.append(0)
  asset['OBV'] = obv
  return asset

'''
Simple Moving Average (SMA)
Source: https://www.tradingview.com/support/solutions/43000696841-simple-moving-average/
Inputs: asset -> The dataframe of the stock
        period -> The period of the simple moving average (default = 20)
'''
def sma(asset, period = 20):
  asset[f'SMA_{period}'] = asset['Close'].rolling(window = period).mean()
  return asset

'''
Exponentially Weighted Moving Average (EWMA)
Source: https://www.investopedia.com/articles/07/ewma.asp
Inputs: asset -> The dataframe of the stock
        period -> The period of the exponentially weighted moving average (default = 20)
'''
def ewma(asset, period = 20):
  asset[f'EWMA_{period}'] = asset['Close'].ewm(span = period).mean()
  return asset

'''
Bollinger Bands
Source: https://www.investopedia.com/trading/using-bollinger-bands-to-gauge-trends/
Inputs: asset -> The dataframe of the stock
        period -> The period of the simple moving average (default = 20)
'''
def bollinger_band(asset, period = 20):
  asset['TP_BB'] = (asset['High'] + asset['Low'] + asset['Adj Close'])/3
  asset['BOLM'] = asset['TP_BB'].rolling(window = period).mean()
  stddev = asset['TP_BB'].rolling(window = period).std()
  asset['BOLU'] = asset['BOLM'] + 2 * stddev
  asset['BOLD'] = asset['BOLM'] - 2 * stddev
  return asset

'''
Keltner Channels
Source: https://www.investopedia.com/terms/k/keltnerchannel.asp
Inputs: asset -> The dataframe of the stock
        period -> The period of the exponentially weighted moving average (default = 20)
'''
def keltner_channel(asset, period = 20):
  asset['KCM'] = asset['Close'].ewm(span = period).mean()
  tr = asset['High'] - asset['Low']
  atr = tr.ewm(span = period).mean()
  asset['KCU'] = asset['KCM'] + 2 * atr
  asset['KCL'] = asset['KCM'] - 2 * atr
  return asset

'''
Relative Strength Indicator (RSI)
Source: https://www.tradingview.com/support/solutions/43000502338-relative-strength-index-rsi/
Inputs: asset -> The dataframe of the stock
        period -> The period for calculating momentum (default = 14)
        ewma -> The exponential weighted moving average is the default setting
'''
def rsi(asset, period = 14, ewma = True):
  close_delta = asset['Close'].diff()
  up = close_delta.clip(lower = 0)
  down = -1 * close_delta.clip(upper = 0)
  if ewma == True:
      ma_up = up.ewm(com = period - 1, adjust = True, min_periods = period).mean()
      ma_down = down.ewm(com = period - 1, adjust = True, min_periods = period).mean()
  else:
      ma_up = up.rolling(window = period, adjust = False).mean()
      ma_down = down.rolling(window = period, adjust = False).mean()

  rsi = ma_up / ma_down
  rsi = 100 - (100/(1 + rsi))
  asset['RSI'] = rsi
  return asset


'\nChaikin Osciilator\nSource:\nInputs:\n'